### Load packages
Please note the use of `datacube.utils` package `geometry`: 
this is important for saving the coordinate reference system of the incoming shapefile in a format that the Digital Earth Australia query can understand.

In [10]:
%matplotlib inline

import datacube
import rasterio.crs
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from datacube.utils import geometry

import sys
sys.path.append('../Scripts')
from dea_plotting import map_shapefile
from dea_temporal import time_buffer
from dea_spatialtools import xr_rasterize

In [2]:
dc = datacube.Datacube(app='Analyse_Ausplots_polygons')

/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


## Analysis parameters

In [3]:
time_of_interest = ("2011-09-01", "2019-10-31")
vector_file = 'Ausplots_polyg.shp'
attribute_col = 'plotID'
products = ['ls7_fc_albers']
measurements = ['BS', 'PV', 'NPV', 'UE']
output_crs = 'EPSG:3577'
align = (0, 0)

### Import the shapefile

In [4]:
gdf = gpd.read_file(vector_file)

### Create a datacube query object
We then create a dictionary that will contain the parameters that will be used to load data from the DEA data cube:

> **Note:** We do not include the usual `x` and `y` spatial query parameters here, as these will be taken directly from each of our vector polygon objects.

In [5]:
query = {'measurements': measurements,
         'time': time_of_interest,
         'output_crs': 'EPSG:3577',
         'align': align,
         }

query

{'measurements': ['BS', 'PV', 'NPV', 'UE'],
 'time': ('2011-09-01', '2019-10-31'),
 'output_crs': 'EPSG:3577',
 'align': (0, 0)}

## Loading satellite data

Here we will iterate through each row of the `geopandas.GeoDataFrame` and load satellite data.

In [ ]:
# Create a csv file to save results 
with open('ausplot_results.csv', 'w') as f:
    f.write('ID,PV_mean,PV_std,PV_count,NPV_mean,NPV_std,NPV_count,BS_mean,BS_std,BS_count\n')

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iterrows():
    
    print(f'Feature: {index + 1}/{len(gdf)}')
    
    # Get site ID
    ID = str(row[attribute_col])
    
    # Extract the feature's geometry as a datacube geometry object
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
    
    # Update the query to include our geopolygon
    query.update({'geopolygon': geom}) 
    
    # Load landsat
    l7fc = dc.load(product=products, **query)

    # Generate a polygon mask to keep only data within the polygon
    mask = xr_rasterize(gdf.iloc[[index]], l7fc)
    
    # Mask dataset to set pixels outside the polygon to `NaN`
    l7fcplots = l7fc.where(mask)
    
    # Select only the nearest date image to each survey
    ds = l7fcplots.interp(time=row.date, method='nearest')
    
    # Obtain mean and std dev of pixels within each Ausplot, ignoring nan and -1 values
    stats = []
    for cover in ['PV', 'NPV', 'BS']:
        d = ds[cover].values
        d = d[(d != -1) & (np.isnan(d) != 1)]
        if d.size > 1:
            stats.append('%.2f'%np.mean(d))
            stats.append('%.2f'%np.std(d))
            stats.append('%i'%d.size)
        else:
            stats.append('No valid pixels')
            stats.append('No valid pixels')
            stats.append('No valid pixels')
        
    # Write to csv
    with open('ausplot_results.csv', 'a') as f:
        f.write('%s,%s\n'%(ID, ','.join(stats)))
    

Feature: 1/539
Feature: 2/539
Feature: 3/539
Feature: 4/539
Feature: 5/539
Feature: 6/539
Feature: 7/539
Feature: 8/539
Feature: 9/539
Feature: 10/539
Feature: 11/539
Feature: 12/539
Feature: 13/539
Feature: 14/539
Feature: 15/539
Feature: 16/539
Feature: 17/539
Feature: 18/539
Feature: 19/539
Feature: 20/539
Feature: 21/539
Feature: 22/539
Feature: 23/539
Feature: 24/539
Feature: 25/539
Feature: 26/539
Feature: 27/539
Feature: 28/539
Feature: 29/539
Feature: 30/539
Feature: 31/539
Feature: 32/539
Feature: 33/539
Feature: 34/539
Feature: 35/539
Feature: 36/539
Feature: 37/539
Feature: 38/539
Feature: 39/539
Feature: 40/539
Feature: 41/539
Feature: 42/539
Feature: 43/539
Feature: 44/539
Feature: 45/539
Feature: 46/539
Feature: 47/539
Feature: 48/539
Feature: 49/539
Feature: 50/539
Feature: 51/539
Feature: 52/539
Feature: 53/539
Feature: 54/539
Feature: 55/539
Feature: 56/539
Feature: 57/539
Feature: 58/539
Feature: 59/539
Feature: 60/539
Feature: 61/539
Feature: 62/539
Feature: 63/539
F